In [6]:
from sklearn import datasets
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
df = datasets.load_boston()


In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.data, df.target, test_size=0.2)

model = LinearRegression()
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.7961239009766314

In [8]:
from joblib import dump, load
dump(model, "../models/boston_housing.joblib")

['../models/boston_housing.joblib']

# Why use ONNX
Tensorflow, keras, pytorch are all similar in that they provide functionality to train ML models. But if you look closely, you will notice that they are different when it comes to fast training or neural network architecture flexibility<br>
Using a unified inference framework in production deployments also separates it from the training stage and lets team use whatever framework they are comfortable with.<br>
So in short, onnx is an open-source project that is designed to boost interoperability between different ML frameworks. A shared model file format provides separation between development and deployment stages, which means ML models can be deployed on a wide scale of hardware devices irrespective of the framework that was used to train the model

In [11]:
# convert to onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt
initial_type = [('float_input', FloatTensorType([102, 13]))]
onx = convert_sklearn(model, initial_types=initial_type)
with open("../models/boston_housing.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [12]:
sess = rt.InferenceSession("boston_housing.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]